In [17]:
import pandas as pd
import numpy as np
import talib as ta

In [54]:
data = pd.read_csv("../price_data.csv").iloc[:,1:]
data.head()

data = data.replace(',','', regex=True)

tens = dict(k=10e3, m=10e6, b=10e9)

volume_cols = [col for col in data.columns if 'Volume' in col]
for col in volume_cols:
    for elem in range(len(data[col])):
        factor, exp = data[col][elem][0:-1], data[col][elem][-1].lower()
        data[col][elem] = int(float(factor) * tens[exp])

data.head()

/var/folders/j2/7j0fctvs2258gcs69mmjmlsr0000gn/T/ipykernel_14473/536629038.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col][elem] = int(float(factor) * tens[exp])


,Date,Open_XRP,High_XRP,Low_XRP,Volume_XRP,Change %_x,returns_XRP,Open_NEO_USD,High_NEO_USD,Low_NEO_USD,...,Low_BTC_USD,Volume_BTC_USD,Change %_y.5,returns_BTC_USD,Open_Litecoin,High_Litecoin,Low_Litecoin,Volume_Litecoin,Change %,returns_Litecoin
0,2022-11-18,0.38379,0.38487,0.37519,3806200000,-0.19%,-0.0019,6.699,6.730,6.562,...,16550.0,18100,-0.43%,-0.0043,62.19,63.82,61.63,8509000,0.14%,0.0014
1,2022-11-17,0.37386,0.38619,0.36936,5452800000,2.36%,0.0236,6.689,6.732,6.520,...,16413.0,14800,1.05%,0.0105,56.92,63.23,56.92,10900000,9.35%,0.0935
2,2022-11-14,0.34082,0.37908,0.32185,9099900000,10.56%,0.1056,6.669,6.815,6.191,...,15829.0,67100,1.73%,0.0173,57.36,59.17,53.61,14600000,-1.29%,-0.0129
3,2022-11-11,0.39463,0.39895,0.36638,6264299999,-2.64%,-0.0264,7.168,7.234,6.539,...,16403.0,80300,-3.04%,-0.0304,60.48,64.21,58.58,19200000,1.60%,0.0160
4,2022-11-10,0.33213,0.40259,0.33066,10700000000,18.86%,0.1886,6.188,7.293,6.103,...,15760.0,207300,10.51%,0.1051,50.62,61.22,50.08,20700000,19.48%,0.1948


In [56]:
# Get columns with open prices
open_cols = [col for col in data.columns if "Open" in col]
high_cols = [col for col in data.columns if "High" in col]
low_cols = [col for col in data.columns if "Low" in col]
vol_cols = [col for col in data.columns if "Volume" in col]
coins = [col.split("_")[1] for col in open_cols]
returns_cols = [col for col in data.columns if "return" in col]

for elem in range(len(open_cols)):

    open = data[open_cols[elem]]
    high = data[high_cols[elem]]
    low = data[low_cols[elem]]
    vol = data[vol_cols[elem]]
    coin = coins[elem]
    
    #overlap studies indicators
    data["SMA_{}".format(coin)] = ta.SMA(open, timeperiod=5)
    data["EMA_{}".format(coin)] = ta.EMA(open, timeperiod=5)
    data["WMA_{}".format(coin)] = ta.WMA(open, timeperiod=5)
    data["DEMA_{}".format(coin)] = ta.DEMA(open, timeperiod=5)
    data["TEMA_{}".format(coin)] = ta.TEMA(open, timeperiod=5)
    data["TRIMA_{}".format(coin)] = ta.TRIMA(open, timeperiod=5)
    data["KAMA_{}".format(coin)] = ta.KAMA(open, timeperiod=5)
    data["MAMA_{}".format(coin)] = ta.MAMA(open, fastlimit=0.5, slowlimit=0.05)[0]
    data["T3_{}".format(coin)] = ta.T3(open, timeperiod=5, vfactor=0)

    #momentum indicators
    data["ADX_{}".format(coin)] = ta.ADX(high, low, open, timeperiod=14)
    data["ADXR_{}".format(coin)] = ta.ADXR(high, low, open, timeperiod=14)
    data["APO_{}".format(coin)] = ta.APO(open, fastperiod=12, slowperiod=26, matype=0)
    data["AROONOSC_{}".format(coin)] = ta.AROONOSC(high, low, timeperiod=14)
    data["BOP_{}".format(coin)] = ta.BOP(open, high, low, open)
    data["CCI_{}".format(coin)] = ta.CCI(high, low, open, timeperiod=14)
    data["CMO_{}".format(coin)] = ta.CMO(open, timeperiod=14)
    data["DX_{}".format(coin)] = ta.DX(high, low, open, timeperiod=14)
    data["MACD_{}".format(coin)] = ta.MACD(open, fastperiod=12, slowperiod=26, signalperiod=9)[0]
    data["MACDEXT_{}".format(coin)] = ta.MACDEXT(open, fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)[0]
    data["MACDFIX_{}".format(coin)] = ta.MACDFIX(open, signalperiod=9)[0]
    data["MFI_{}".format(coin)] = ta.MFI(high, low, open, vol, timeperiod=14)
    data["MINUS_DI_{}".format(coin)] = ta.MINUS_DI(high, low, open, timeperiod=14)

    #volatility indicators
    data["ATR_{}".format(coin)] = ta.ATR(high, low, open, timeperiod=14)
    data["NATR_{}".format(coin)] = ta.NATR(high, low, open, timeperiod=14)
    data["TRANGE_{}".format(coin)] = ta.TRANGE(high, low, open)

    #volume indicators
    data["AD_{}".format(coin)] = ta.AD(high, low, open, vol)
    data["ADOSC_{}".format(coin)] = ta.ADOSC(high, low, open, vol, fastperiod=3, slowperiod=10)
    data["OBV_{}".format(coin)] = ta.OBV(open, vol)

    #price transform
    data["AVGPRICE_{}".format(coin)] = ta.AVGPRICE(open, high, low, open)
    data["MEDPRICE_{}".format(coin)] = ta.MEDPRICE(high, low)
    data["TYPPRICE_{}".format(coin)] = ta.TYPPRICE(high, low, open)
    data["WCLPRICE_{}".format(coin)] = ta.WCLPRICE(high, low, open)

    #cycle indicators
    data["HT_DCPERIOD_{}".format(coin)] = ta.HT_DCPERIOD(open)
    data["HT_DCPHASE_{}".format(coin)] = ta.HT_DCPHASE(open)
    data["HT_PHASOR_{}".format(coin)] = ta.HT_PHASOR(open)[0]
    data["HT_SINE_{}".format(coin)] = ta.HT_SINE(open)[0]
    data["HT_TRENDMODE_{}".format(coin)] = ta.HT_TRENDMODE(open)

    #pattern recognition
    data["CDL2CROWS_{}".format(coin)] = ta.CDL2CROWS(open, high, low, open)
    data["CDL3BLACKCROWS_{}".format(coin)] = ta.CDL3BLACKCROWS(open, high, low, open)
    data["CDL3INSIDE_{}".format(coin)] = ta.CDL3INSIDE(open, high, low, open)
    data["CDL3LINESTRIKE_{}".format(coin)] = ta.CDL3LINESTRIKE(open, high, low, open)

    #add previous returns
    data["1Lag_RET_{}".format(coin)] = data[open_cols[elem]].shift(1)
    data["2Lag_RET_{}".format(coin)] = data[open_cols[elem]].shift(2)
    data["3Lag_RET_{}".format(coin)] = data[open_cols[elem]].shift(3)
    data["4Lag_RET_{}".format(coin)] = data[open_cols[elem]].shift(4)
    data["5Lag_RET_{}".format(coin)] = data[open_cols[elem]].shift(5)

    data["Log_RET_{}".format(coin)] = np.log(data[returns_cols[elem]] + 1)

    

/var/folders/j2/7j0fctvs2258gcs69mmjmlsr0000gn/T/ipykernel_14473/1193899632.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["MACDFIX_{}".format(coin)] = ta.MACDFIX(open, signalperiod=9)[0]
/var/folders/j2/7j0fctvs2258gcs69mmjmlsr0000gn/T/ipykernel_14473/1193899632.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["MFI_{}".format(coin)] = ta.MFI(high, low, open, vol, timeperiod=14)
/var/folders/j2/7j0fctvs2258gcs69mmjmlsr0000gn/T/ipykernel_14473/1193899632.py:41: PerformanceWarning: DataFrame is highly fragmente

In [57]:
np.shape(data)

(1416, 690)

In [59]:
data.to_csv("../features.csv")

In [65]:
TRAIN_SIZE = 0.9
VAL_SIZE = 0.05

n_train = int(len(data) * TRAIN_SIZE)
n_val = int(len(data) * VAL_SIZE)

df_train = data.iloc[0:n_train]
df_val = data.iloc[n_train:n_train+n_val]
df_test = data.iloc[n_train+n_val:]

df_train.to_csv("../features_train.csv")
df_val.to_csv("../features_val.csv")
df_test.to_csv("../features_test.csv")

In [67]:
np.shape(df_test)

(72, 690)